## Getting Started with the Maxar Geospatial Platform Tasking API

This sample workflow will guide a user through using the Maxar Geospatial Platform tasking API with real world examples. The process will go through:
* Viewing pipelines
* Creating a tasking request
* Viewing a tasking request
* Canceling a tasking request

#### Authentication
For a more detailed view of how to authorize with MGP, please see the *Auth and Token Service* notebook in this collection.

In [ ]:
import requests
import json

username = "your.email@address.com"
password= "yourpassword"

url = "https://account.maxar.com/auth/realms/mds/protocol/openid-connect/token"

payload = 'client_id=mgp&username={}&password={}&grant_type=password'.format(username, password)
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

response = requests.request("POST", url, headers=headers, data=payload)

access_token = {'Content-Type': 'application/json', 'Authorization': 'Bearer {}'.format(response.json()['access_token'])}

#### Getting a Pipeline

Firstly, a pipeline will need to be found to work with as well as the necessary information needed for that pipeline. If a user is unfamiliar with a pipeline and what is needed for the order to work, the user can use the following request to determine what information needs to be passed in.

In this case, we want to use the *map-ready* pipeline with the namespace of *imagery*. With our pipeline identified, we can narrow down the response of the request to show us the necessary information needed to place a tasking request

In [ ]:
get_pipeline_url = "https://api.maxar.com/ordering/v1/pipelines/imagery/map-ready"

get_pipeline_response = requests.request("GET", get_pipeline_url, headers=access_token)

required_info = get_pipeline_response.json()['data']['settings_schema']['required']
print("The required information needed to place an order on this pipeline is: {}".format(required_info))
for info in required_info:
    type_info = get_pipeline_response.json()['data']['settings_schema']['properties'][info]
    print("The type of data for {} is: {}".format(info, type_info))

#### Create a Tasking Request

Before a tasking request can be created, the datetime module will need to be imported to make it easier to get UTC dates in `ISO-8601` format. Tasking requests must be made at least 5 minutes into the future.

In [ ]:
import datetime

now_start = datetime.datetime.utcnow() + datetime.timedelta(minutes=10)
now_time = now_start.isoformat()
end_start = datetime.datetime.utcnow() + datetime.timedelta(days=1)
end_time = end_start.isoformat()
now = now_time.replace(now_time[-7:], "+00:00")
end = end_time.replace(now_time[-7:], "+00:00")
print(now)
print(end)

Having set the beginning and ending times, a user will now need to set up a tasking template. Using the information gathered from the pipeline request, the user can construct a template. *NOTE: The `inventory_ids` field needs to be included in the tasking request, but no ids need to be passed in. Instead, use the string shown in the list as a placeholder.*

In [ ]:
settings = {"inventory_ids": ["$.id"], "customer_description": "Your description"}
output_config = {"amazon_s3": {"bucket": "YourS3BucketName", "prefix": "Your/S3/bucket/prefix/name"}}
notifications = {"type": "email", "target": "your.email@address.com", "level": "INITIAL_FINAL"}

In [ ]:
template = [{
    "pipeline": "imagery/map-ready",
    "template": {
        "settings": settings,
        "output_config": output_config,
        "notifications": [notifications],
        "metadata": {
            "project_id": "Your tasking name"
        }
    }
}]

A geoJSON will also be set in advance to use in the tasking request.

In [ ]:
coordinates = {
    "type": "Polygon", 
    "coordinates": [
        [
            [152.931404, -27.556982],
            [153.172073, -27.556982],
            [153.172073, -27.394936],
            [152.931404, -27.394936],
            [152.931404, -27.556982]
        ]
    ]
}

With the variables set, the tasking request is almost ready to submit. First, run a validation on the tasking request to make sure that everything is formatted correctly. `50cm_Color` will be the recipe used for this example.

In [ ]:
tasking_validate_url = "https://api.maxar.com/tasking/v1/tasking?validation_only=true"

tasking_validate_payload = json.dumps({"start_datetime": now, "end_datetime": end, "aoi_geojson": coordinates, 
                                       "recipe": "50cm_Color", "order_templates": template})

tasking_validate_response = requests.request("POST", tasking_validate_url, headers=access_token, data=tasking_validate_payload)

print(tasking_validate_response.json())

With the validaton passing, the full tasking request can be submitted.

In [ ]:
tasking_url = "https://api.maxar.com/tasking/v1/tasking"

tasking_payload = json.dumps({"start_datetime": now, "end_datetime": end, "aoi_geojson": coordinates, 
                              "recipe": "50cm_Color", "order_templates": template})

tasking_response = requests.request("POST", tasking_url, headers=access_token, data=tasking_payload)

print(tasking_response.json())
tasking_id = tasking_response.json()['data']['id']

#### Viewing Tasking Requests

The tasking request has been submitted and the `tasking_id` has been set to a variable for use further along in the workflow. With the request submitted, a user can now check on the status of the request.

In [ ]:
get_tasking_url = "https://api.maxar.com/tasking/v1/tasking/{}".format(tasking_id)

get_tasking_response = requests.request("GET", get_tasking_url, headers=access_token)

print(get_tasking_response.json())

#### Cancel a Tasking Request

In some cases, a user may want to cancel a tasking order before it triggers. This can be acomplished with the following request.

In [ ]:
cancel_tasking_url = "https://api.maxar.com/tasking/v1/tasking/{}/cancel".format(tasking_id)

cancel_tasking_payload = json.dumps({"reason": "Your reason for cancelling"})

cancel_tasking_response = requests.request("POST", cancel_tasking_url, headers=access_token, data=cancel_tasking_payload)

print(cancel_tasking_response.json())